### Testing and Validation of a network with Batfish

In this notebook we will focus on different types of validations that can be done on the results returned from Batfish. We will see how these validations are possible in Python using simple and easy to use [Pandas APIs](https://pandas.pydata.org/pandas-docs/stable/).

In [1]:
%run startup.py

/Users/harshverma/sprints/open_source/pybatfish/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/harshverma/sprints/open_source/pybatfish/pybatfish/client/commands.py:49: UserWarning: Pybatfish public API is being updated, note that API names and parameters will soon change.
  "Pybatfish public API is being updated, note that API names and parameters will soon change.")


### Initializing our Network and Snapshot

In [2]:
NETWORK_NAME = "example_network"
SNAPSHOT_NAME = "example_snapshot"
SNAPSHOT_PATH = "../test_rigs/example"

try:
    bf_init_network(NETWORK_NAME)
except BatfishException as e:
    bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME)

'{\n  "answerElements" : [\n    {\n      "class" : "org.batfish.datamodel.answers.InitInfoAnswerElement",\n      "parseStatus" : {\n        "as1border1" : "PASSED",\n        "as1border2" : "PASSED",\n        "as1core1" : "PASSED",\n        "as2border1" : "PASSED",\n        "as2border2" : "PASSED",\n        "as2core1" : "PASSED",\n        "as2core2" : "PASSED",\n        "as2dept1" : "PASSED",\n        "as2dist1" : "PASSED",\n        "as2dist2" : "PASSED",\n        "as3border1" : "PASSED",\n        "as3border2" : "PASSED",\n        "as3core1" : "PASSED",\n        "host1" : "PASSED",\n        "host2" : "PASSED",\n        "iptables/host1.iptables" : "PASSED",\n        "iptables/host2.iptables" : "PASSED"\n      }\n    }\n  ],\n  "status" : "SUCCESS",\n  "summary" : {\n    "numFailed" : 0,\n    "numPassed" : 0,\n    "numResults" : 0\n  }\n}\n'

### NTP servers validation

As a first step, let's get all the NTP servers on the border routers.

In [3]:
ntp_servers = bfq.nodeProperties(nodeRegex=".*border.*", propertySpec="ntp-servers").answer().frame()
ntp_servers

,node,ntp-servers
0,as1border1,[]
1,as1border2,"[18.18.18.18, 23.23.23.23]"
2,as2border1,"[18.18.18.18, 23.23.23.23]"
3,as2border2,[18.18.18.18]
4,as3border1,"[18.18.18.18, 23.23.23.23]"
5,as3border2,"[18.18.18.18, 23.23.23.23]"


#### Validation Scenario 1
All nodes under consideration should have **18.18.18.18** and **23.23.23.23** in their set of NTP servers.

Let's get a list of all nodes conforming and violating this condition.

In [4]:
# Conformers (Nodes which have both 18.18.18.18 and 23.23.23.23 in their set of NTP servers)
ns_conformers = ntp_servers[ntp_servers["ntp-servers"].apply(lambda x:set(['18.18.18.18','23.23.23.23']).issubset(set(x)))]
ns_conformers

,node,ntp-servers
1,as1border2,"[18.18.18.18, 23.23.23.23]"
2,as2border1,"[18.18.18.18, 23.23.23.23]"
4,as3border1,"[18.18.18.18, 23.23.23.23]"
5,as3border2,"[18.18.18.18, 23.23.23.23]"


In [5]:
# Violators (Nodes which don't have both 18.18.18.18 and 23.23.23.23 in their set of NTP servers)
ns_violators = ntp_servers[ntp_servers["ntp-servers"].apply(lambda x:not set(['18.18.18.18','23.23.23.23']).issubset(set(x)))]
ns_violators

,node,ntp-servers
0,as1border1,[]
3,as2border2,[18.18.18.18]


#### Validation Scenario 2
All nodes under consideration should have NTP servers only from the set of **18.18.18.18** and **23.23.23.23**.

In [6]:
# Conformers (Nodes whose NTP servers are from the set of 18.18.18.18 and 23.23.23.23)
ns_conformers = ntp_servers[ntp_servers["ntp-servers"].apply(lambda x:set(['18.18.18.18','23.23.23.23']).issuperset(set(x)))]
ns_conformers

,node,ntp-servers
0,as1border1,[]
1,as1border2,"[18.18.18.18, 23.23.23.23]"
2,as2border1,"[18.18.18.18, 23.23.23.23]"
3,as2border2,[18.18.18.18]
4,as3border1,"[18.18.18.18, 23.23.23.23]"
5,as3border2,"[18.18.18.18, 23.23.23.23]"


This will also return the nodes which have an empty set of NTP servers (technically they do not violate our condition). But if we want to exclude them, we can add a condition in the above command to filter out empty sets of NTP servers.

In [7]:
ns_conformers = ntp_servers[ntp_servers["ntp-servers"].apply(lambda x:set(['18.18.18.18','23.23.23.23']).issuperset(set(x)) and bool(x))]
ns_conformers

,node,ntp-servers
1,as1border2,"[18.18.18.18, 23.23.23.23]"
2,as2border1,"[18.18.18.18, 23.23.23.23]"
3,as2border2,[18.18.18.18]
4,as3border1,"[18.18.18.18, 23.23.23.23]"
5,as3border2,"[18.18.18.18, 23.23.23.23]"


In [8]:
# Violators (Nodes whose NTP servers are not from the set of 18.18.18.18 and 23.23.23.23)
ns_violators = ntp_servers[ntp_servers["ntp-servers"].apply(lambda x:not set(['18.18.18.18','23.23.23.23']).issuperset(set(x)))]
ns_violators

,node,ntp-servers


The above query will not return any result, as in this case there are no violators.

#### Validation Scenario 3
All nodes should have set of NTP servers that is equal to the set of **18.18.18.18** and **23.23.23.23**

In [9]:
# Conformers (Nodes whose NTP servers are equal to the set of 18.18.18.18 and 23.23.23.23)
ns_conformers = ntp_servers[ntp_servers["ntp-servers"].apply(lambda x:set(['18.18.18.18','23.23.23.23']) == set(x))]
ns_conformers

,node,ntp-servers
1,as1border2,"[18.18.18.18, 23.23.23.23]"
2,as2border1,"[18.18.18.18, 23.23.23.23]"
4,as3border1,"[18.18.18.18, 23.23.23.23]"
5,as3border2,"[18.18.18.18, 23.23.23.23]"


In [10]:
# Violators (Nodes whose set of NTP servers is not equal to the set of 18.18.18.18 and 23.23.23.23)

In [11]:
ns_violators = ntp_servers[ntp_servers["ntp-servers"].apply(lambda x:set(['18.18.18.18','23.23.23.23']) != set(x))]
ns_violators

,node,ntp-servers
0,as1border1,[]
3,as2border2,[18.18.18.18]


To find out which NTP servers actually differed from our expected set of NTP servers, we can do a set difference for each row

In [12]:
ns_difference = ntp_servers["ntp-servers"].map(lambda x: set(['18.18.18.18','23.23.23.23']) - set(x))
ns_difference

0    {18.18.18.18, 23.23.23.23}
1                            {}
2                            {}
3                 {23.23.23.23}
4                            {}
5                            {}
Name: ntp-servers, dtype: object

That's it for now!